# 第5章 機械学習を用いた因果推論

#### ・本章では、非線形で変数間に相互作用のある因果関係における処置の効果の推定手法に関して、機械学習を用いた因果推論について取り扱う .

## 5-1 ランダムフォレストによる分類と回帰のしくみ

## 決定木による分類

#### ・決定木分析 : データを基に決定木と呼ばれる樹形図を作成し、分析を行う方法のこと .

#### ・今回構築したいモデルは、とある「iさん」が[テレビCMを見た]かどうか、すなわち処置を受けたかどうかを示す値$Z^i$が0なのか1なのかを分類予測する機械学習モデル .

#### これは第4章で解説した傾向スコアを求めるロジスティック回帰の代替となる .

#### ・予測したい変数$Z$を目的変数、もしくは被説明変数と呼び、今回予測に使用するのは年齢変数$x_1$と性別変数$x_2$とする .

#### この予測に使用する変数を特徴量、もしくは説明変数と呼ぶ .

#### ・決定木では図5.1.1のような「説明変数の値に対する条件式」を構築し、各データがその条件式に対して、YesなのかNoなのかに応じて分岐させる .

#### この条件式での判定、分岐を繰り返し、最後に変数$Z$が1なのか0なのかを決定する .

#### 分岐させた先を「ノード」という .

#### ・図5.1.1のように、決定木はif文の連続で記載されるルールベースの手法となる .　ただし、この条件式のルールを作成する際に工夫が行われる .

![alt text](pict1.png)

#### ・分類の決定木において条件式を作成するにあたり、「ジニ不純度」と呼ばれる指標を使用する .　ジニ不純度$I(t)$を式で表すと、